In [ ]:
import nltk 
import re
from nltk.stem import PorterStemmer
import os 
import json
from charset_normalizer import from_path
STEMMER = PorterStemmer()

## I. Read the files and build two large consolidate files that are the union of all the documents in 20N and BAC.

In [20]:
os.getcwd()

'/home/erich/dev/Curso_NLP/Tarea 2'

In [51]:
ACTUAL_PATH = os.getcwd()
PATH_20N = os.path.join(ACTUAL_PATH, "data/20news-18828")
PATH_BAC = os.path.join(ACTUAL_PATH, "data/BAC/blogs")
PATH_FINAL_FILES = os.path.join(ACTUAL_PATH,"data/final_files")

### UPLOAD_FILES

#### UPLOADING 20N 

In [ ]:
NEW_20N_FILE = os.path.join(PATH_FINAL_FILES, "20N.jsonl")

mayor_folders_20N = os.listdir(PATH_20N)
dictionary = {}

with open(NEW_20N_FILE, "w", encoding="utf-8", errors="replace") as f_n:
    for folder in mayor_folders_20N:
        minor_files_path = os.path.join(PATH_20N, folder)
        minor_files = os.listdir(minor_files_path)
        for file in minor_files:
            file_path = os.path.join(minor_files_path, file)
            with open(file_path, "r", encoding="utf-8", errors="replace") as f:
                text = f.read().strip()

            record = {
                "id": file,
                "theme": folder,
                "source": "20N",
                "text": text
            }
            unit = folder + file
            if file in dictionary.keys():
                dictionary[unit] += 1
            else: 
                dictionary[unit] = 1
            f_n.write(json.dumps(record, ensure_ascii=False) + "\n")

In [ ]:
## TODO: Verificar si los encodings estan bien

#### UPLOADING BAC

In [ ]:
NEW_BAC_FILE = os.path.join(PATH_FINAL_FILES, "BAC.jsonl")
mayor_folders_BAC = os.listdir(PATH_BAC)
with open(NEW_BAC_FILE, "w", encoding="utf-8", errors="replace") as f_n:
    for file in mayor_folders_BAC:
        post_num = 0
        file_path = os.path.join(PATH_BAC, file)
        with open(file_path, "r", encoding="utf-8", errors="replace") as f:
            text = f.read().strip()
            text = re.sub(r"</?Blog>", "", text)
        post_list = text.split("<post>")
        for post in post_list:
            post = post.strip().replace("</post>", "")
            record = {
                "id": file,
                "post_num": post_num,
                "source": "BAC",
                "text": post
            }
            f_n.write(json.dumps(record, ensure_ascii=False) + "\n")
            post_num +=1

## II. Tokenize by sentence

In [17]:
def preprocess_text(text:str) -> list[str]:

    text = text.lower()
    text = re.sub(r"[^a-zA-Z1-9\s]", " ", text, flags=re.I | re.A | re.MULTILINE)
    text = re.sub(r"\d", "NUM", text)
    text = "<s> " + text
    text = text + " </s>"
    text = text.strip().split()
    tokens = [STEMMER.stem(word) for word in text]
    return tokens
    
def get_sentences(text:str, k: int) -> list[list]:
    sentences_list = []
    text = text.split()
    last = 0
    for i in range(k,len(text)+1):
        sentences = text[last:i]
        last +=1
        text_slice = " ".join(sentences)
        text_slice_token = preprocess_text(text_slice)
        sentences_list.append(text_slice_token)
    return sentences_list
    

In [12]:
preprocess_text("hello how are you playing pong and pong your mom")

['<s>',
 'hello',
 'how',
 'are',
 'you',
 'play',
 'pong',
 'and',
 'pong',
 'your',
 'mom',
 '</s>']

In [18]:
get_sentences("hello how are you playing pong and pong your mom", k=2)

[['<s>', 'hello', 'how', '</s>'],
 ['<s>', 'how', 'are', '</s>'],
 ['<s>', 'are', 'you', '</s>'],
 ['<s>', 'you', 'play', '</s>'],
 ['<s>', 'play', 'pong', '</s>'],
 ['<s>', 'pong', 'and', '</s>'],
 ['<s>', 'and', 'pong', '</s>'],
 ['<s>', 'pong', 'your', '</s>'],
 ['<s>', 'your', 'mom', '</s>']]